In [1]:
import json
import numpy as np
import pandas as pd
import math
import torch

In [2]:
path_to_data="/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/data/data.json"
path_to_labels = "/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/data/data.info"

In [3]:
# To be able to import the file
import sys
pathname="/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/scripts"
if pathname not in sys.path:
    sys.path.append(pathname)

In [4]:
import getData as gd
import importlib
importlib.reload(gd)

<module 'getData' from '/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/scripts/getData.py'>

In [5]:
getDat = gd.getData(path_to_data=path_to_data, path_to_labels=path_to_labels)

In [6]:
k_mers = getDat.get_unique_kmers()
labels = getDat.get_labels()
df = getDat.get_data(num_entries=10)
df = pd.merge(df, labels, how="left", left_on = ("transcript", "position"), 
              right_on = ("transcript_id", "transcript_position"))
df = df[['gene_id', 'transcript', 'position', "k-mer bases", "values", "label"]]
# df[['k-1', 'k', 'k+1']]=df.apply(lambda x : [x['k-mer bases'][i:i+5] for i in range(3)], axis=1, result_type="expand")

In [7]:
df

,gene_id,transcript,position,k-mer bases,values,label
0,ENSG00000004059,ENST00000000233,244,AAGACCA,"[[0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0...",0
1,ENSG00000004059,ENST00000000233,261,CAAACTG,"[[0.0126, 1.95, 111.0, 0.0125, 1.27, 108.0, 0....",0
2,ENSG00000004059,ENST00000000233,316,GAAACAG,"[[0.00432, 2.02, 104.0, 0.00299, 3.56, 99.3, 0...",0
3,ENSG00000004059,ENST00000000233,332,AGAACAT,"[[0.0134, 4.71, 132.0, 0.00447, 4.24, 98.8, 0....",0
4,ENSG00000004059,ENST00000000233,368,AGGACAA,"[[0.015, 6.97, 118.0, 0.0106, 3.04, 123.0, 0.0...",0
5,ENSG00000004059,ENST00000000233,404,AGAACAC,"[[0.00444, 5.4, 131.0, 0.00664, 7.26, 102.0, 0...",0
6,ENSG00000004059,ENST00000000233,431,TGGACAG,"[[0.0102, 2.78, 118.0, 0.00697, 6.35, 119.0, 0...",0
7,ENSG00000004059,ENST00000000233,440,ATGACCG,"[[0.00432, 2.94, 97.5, 0.00531, 11.0, 120.0, 0...",0
8,ENSG00000004059,ENST00000000233,471,TGAACTC,"[[0.00266, 10.6, 125.0, 0.00681, 2.87, 104.0, ...",0
9,ENSG00000004059,ENST00000000233,539,AGGACAT,"[[0.017, 6.09, 115.0, 0.00864, 5.81, 120.0, 0....",0


In [14]:
train_genes = prepD.splitdata(label_df=labels)

In [15]:
len(train_genes)

2602

# Data Loader

In [92]:
import prepData as prepD
import importlib
importlib.reload(prepD)

<module 'prepData' from '/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/scripts/prepData.py'>

In [93]:
prepData = prepD.prepData(train_genes=None, path_to_data=path_to_data, path_to_labels=path_to_labels)

In [94]:
dataloader = prepData.get_data_loader()

[1, 21]


In [95]:
np.min([1.0, 20])

1.0

In [99]:
x

50 50


In [63]:
sample_weights = prepData.get_data_loader()

In [68]:
q = np.array(sample_weights)
len(q[q==23])

204

In [67]:
sizes

[4796, 204]

In [72]:
dataloader = torch.utils.data.DataLoader(dataset= prepData, batch_size=5)
dataiter = iter(dataloader)
data= dataiter.next()

In [73]:
features, label = data

In [74]:
q=nn.BCELoss()

In [31]:
net = m6aNet(5, 20)
res = net.forward(features)

In [56]:
q = np.array(prepData.df['label'])
q = np.array([1, 2, 3, 4])

In [57]:
list(map(lambda x : x+1, q))

[2, 3, 4, 5]

In [26]:
import torch
import torch
import torch.nn as nn

In [30]:
class m6aNet(nn.Module):
    def __init__(self, batchsize, readsize):
        self.batchsize = batchsize
        self.readsize = readsize
        super(m6aNet, self).__init__()
        # Embedding Layer
        self.embed = nn.Embedding(66, 2)

        ## First Layer ##
        self.read_level_prob_1 = nn.Linear(15, 150)
        # First Batch Norm Layer
        self.norm_1 = nn.BatchNorm1d(num_features=150)
        # First Activation Layer
        self.activ_1=nn.ReLU()
        # First Dropout Layer
        self.drop_1 = nn.Dropout(p=0.00)

        ## Second Layer ##
        self.read_level_prob_2 = nn.Linear(150, 32)
        # Second Activation Layer
        self.activ_2=nn.ReLU()
        # Second Dropout Layer
        self.drop_2 = nn.Dropout(p=0.00)

        ## Third Layer ##
        self.read_level_prob_3 = nn.Linear(32, 1)
        # Sigmoid Activation
        self.sig_1 = nn.Sigmoid()

    
    
    def forward(self, x):
        ### X is a tensor of shape (batchsize, readsize=20, 12) ###
        
        # Extract numeric features        
        numerics = x[:, :, :9]
        # # Extract Bases
        bases = x[:, :, 9:].type(torch.int64)
        # # Feed to embedding layer
        bases = self.embed(bases)

        # Reshape
        bases = bases.reshape(self.batchsize, self.readsize, 3*2)
        # Combine embedded output with numeric features
        x = torch.concat((numerics, bases), 2).type(torch.float)

        #### Feed Forward  ####

        ## First Layer ##
        x = self.read_level_prob_1(x)
        # First Batch Norm Layer
        x = x.transpose(dim0=1, dim1=2) # Need to transpose first
        x = self.norm_1(x)
        x = x.transpose(dim0=1, dim1=2) # Then transpose back
        # First Activation Layer
        x= self.activ_1(x)
        # First Dropout Layer
        x = self.drop_1(x)

        ## Second Layer ##
        x = self.read_level_prob_2(x)
        # Second Activation Layer
        x = self.activ_2(x)
        # Second Dropout Layer
        x = self.drop_2(x)

        ## Third Layer ##
        x = self.read_level_prob_3(x)
        # Sigmoid Activation
        x = self.sig_1(x)
        x = x.reshape(-1, self.readsize)
        
        # Final Output
        r = 1 - torch.prod(1 - x, axis=1)
        return r

            
        
        
        
        
        

In [204]:
torch.from_numpy(np.array(int(df.iloc[0, -1])))

tensor(0)

In [201]:
df.dtypes

gene_id        object
transcript     object
position        int64
k-mer bases    object
values         object
label          object
dtype: object

In [34]:
import pickle

In [99]:
labels.dtypes

0
gene_id                object
transcript_id          object
transcript_position     int64
label                   int64
dtype: object

In [98]:
labels['label'] = labels['label'].astype(int)

In [13]:
from torch.utils.data import dataloader